In [98]:
import math
from collections import defaultdict
import random

In [99]:
class Dataset:
    def __init__(self, images_file, labels_file, lines_per_image = 28):
        self.images = []
        self.labels = []
        self.counts_by_label = defaultdict(int)
        self.priors = {}
        with open(images_file) as file:
            EOF = False
            while not EOF:
                image = []
                for i in range(lines_per_image):
                    line = file.readline()
                    if not line:
                        EOF = True
                        break
                    image.append(list(line))
                if EOF:
                    break
                self.images.append(image)
        with open(labels_file) as file:
            for i,label in enumerate(file):
                label = int(label)
                self.labels.append(label)
                self.counts_by_label[label]+=1
    def display(self, i):
        print("".join(map(lambda x: "".join(x),self.images[i])))
    def __len__(self):
        return len(self.labels)
    
    def shuffleData(self):
        order = list(zip(self.labels, self.images))
        random.shuffle(order)
        tempL, tempI = zip(*order) #zip turns them into giant tuples, want in list form
        self.labels = list(tempL)
        self.images = list(tempI)
    
    
class Perceptron:
    def __init__(self, label):
        self.label = label                                
        self.bias = 0 
        self.weightVector = [[int(random.random() *10) for i in range(28)] for j in range(28)] 
        self.totalCount = 0
        self.decayed = 0
        
    def display(self):
        print('\n'.join(str(self.weightVector[i]) for i in range(28)))
        
    def updateBias(self):
        self.bias = self.totalCount
    
    def getDecay(self, totalImages):
        return int(totalImages/2000)/10.0
    
    def trainVectorOnCorrect(self, image, totalCount): 
        for i in range(28):
            for j in range(28):
                if(image[i][j] != ' ' and image[i][j] != '\n'):
                    self.weightVector[i][j] += (2 - self.getDecay(totalCount)) #alpha = 2-learningRateDecay seems to work best
    def trainVectorOnIncorrect(self, image, totalCount):
        for i in range(28):
            for j in range(28):
                if(image[i][j] != ' ' and image[i][j] != '\n'):
                     self.weightVector[i][j] -= (2 - self.getDecay(totalCount)) 
                        
    def imageEvaluation(self, image):
        likelihood = 0
        for i in range(28):
            for j in range(28):
                if(image[i][j] == ' '):
                    likelihood -= self.weightVector[i][j]
                elif image[i][j] != '\n':
                    likelihood += self.weightVector[i][j]
        return likelihood

In [100]:
def retrainLoop(image, actualLabel, totalCount):
    iterationLimit = 10
    while(True):
        if(iterationLimit < 0):
            break
        chances = [perceptrons[j].imageEvaluation(image) for j in range(10)]
        bestGuess = chances.index(max(chances))
        #print(chances)
        #print("is: " +  str(actualLabel) + " guessed: " + str(bestGuess))
        if bestGuess == actualLabel:
            perceptrons[actualLabel].trainVectorOnCorrect(image, totalCount)
            break
        else :
            perceptrons[actualLabel].trainVectorOnIncorrect(perceptrons[bestGuess].weightVector, totalCount)
            perceptrons[actualLabel].trainVectorOnCorrect(image, totalCount)
        iterationLimit -= 1

In [112]:
trainingData = Dataset("trainingimages", "traininglabels")
testData = Dataset("testimages", "testlabels")

In [113]:
perceptrons = [None]*10
for i in range(10):
    perceptrons[i] = Perceptron(i)


#
# RUNNING ALL EPOCHS TAKES A VERY LONG TIME
#
epochs = 4
totalImages = 0
for q in range(epochs):
    correctGuesses = 0
    trainingData.shuffleData()
    for l in range(10): 
        perceptrons[l].updateBias()
    
    for i in range(len(trainingData.images)):
        totalImages +=1
        currImage = trainingData.images[i]
        actualLabel = trainingData.labels[i]
        for j in range(10):
            activate = perceptrons[j].imageEvaluation(currImage) + perceptrons[j].bias
            if activate > ((q+1)* 1000): 
                if j == actualLabel :
                    correctGuesses +=1
                    perceptrons[actualLabel].totalCount +=1
                    perceptrons[actualLabel].trainVectorOnCorrect(currImage, totalImages)
                else :
                    retrainLoop(currImage, actualLabel, 0)
            elif j== actualLabel:
                perceptrons[actualLabel].totalCount +=1
                perceptrons[actualLabel].trainVectorOnCorrect(currImage, totalImages)

    print("epoch number: " + str(q) + ", accuracy: " + str(correctGuesses/len(trainingData.images)))

epoch number: 0, accuracy: 0.936
epoch number: 1, accuracy: 1.0
epoch number: 2, accuracy: 1.0
epoch number: 3, accuracy: 1.0


In [114]:

confusion_matrix_count = [[0 for i in range(10)] for j in range(10)]
totalImages = 0
indices = len(testData.images)
for i in range(indices):
    totalImages +=1
    currImage = testData.images[i]
    chances = [perceptrons[j].imageEvaluation(currImage) for j in range(10)]
    bestGuess = chances.index(max(chances))
    actualLabel = testData.labels[i]
    if bestGuess == actualLabel :
        perceptrons[actualLabel].trainVectorOnCorrect(currImage, totalImages)
        confusion_matrix_count[bestGuess][bestGuess] +=1
    else :
        retrainLoop(currImage, actualLabel, totalImages)
        confusion_matrix_count[actualLabel][bestGuess] +=1


In [115]:
for i in range(10):
    print(confusion_matrix_count[i])
setTotal = 0
setCorrect = 0
for i in range(10):
    correct = confusion_matrix_count[i][i]
    total = sum(confusion_matrix_count[i])
    rate = 100* correct/total
    print("Label: "+ str(i) + " was accurate at a rate of: " + str(rate) + "%" )
    setCorrect += correct
    setTotal += total
setRate = 100* setCorrect/setTotal    
print("overall accuracy: " + str(setRate) + "% \n")
print("Perceptrons and their respective weight vectors were reset in between all the below test: \n")
print("no decay, I=0, B=0, NW, FO, #E=2 -> overall accuracy: 78.0%")
print("no decay, I=0, B=0, NW, FO, #E=3 -> overall accuracy: 78.9%")
print("no decay, I=0, B=0, NW, FO, #E=4 -> overall accuracy: 77.3%")
print("no decay, I=0, B=0, NW, SO, #E=2 -> overall accuracy: 76.1%")
print("no decay, I=0, B=0, NW, SO, #E=3 -> overall accuracy: 79.0%")
print("no decay, I=0, B=0, NW, SO, #E=4 -> overall accuracy: 77.0%")
print("no decay, I=0, B=0, WWC, SO, #E=2 -> overall accuracy: 79.5%")
print("no decay, I=0, B=0, WWC, SO, #E=3 -> overall accuracy: 79.3%")
print("no decay, I=0, B=0, WWC, SO, #E=4 -> overall accuracy: 79.8%")
print("no decay, I=0, B=SF, WWC, SO, #E=2 -> overall accuracy: 78.8%")
print("no decay, I=0, B=SF, WWC, SO, #E=3 -> overall accuracy: 80.1%")
print("no decay, I=0, B=SF, WWC, SO, #E=4 -> overall accuracy: 81.0%")
print("no decay, I=R, B=SF, WWC, SO, #E=2 -> overall accuracy: 79.0%")
print("no decay, I=R, B=SF, WWC, SO, #E=3 -> overall accuracy: 80.0%")
print("no decay, I=R, B=SF, WWC, SO, #E=4 -> overall accuracy: 77.8%")
print("with decay, I=R, B=SF, WWC, SO, #E=2 -> overall accuracy: 77.5%")
print("with decay, I=R, B=SF, WWC, SO, #E=3 -> overall accuracy: 77.6%")
print("with decay, I=R, B=SF, WWC, SO, #E=4 -> overall accuracy: 78.8%")

print("\nones above here had retrain loop iteration limit set to 10, the ones below have it set to 2\n")

print("Key: \nI=initial value for spots in the weight vector, with 0->inital values all 0, and R-> random, 0 through 9, inital values \nB=bias added to activation check for perceptrons, with 0->bias value is 0, and SF->instances of digit so far\nNW: when testing no changing the weight vector of a perceptron; WWC: when testing update weight vectors of perceptrons\nFO: fixed order for training set; SO: randomly shuffled order for training set\n#E=number of epochs the training set was run for")

[71, 0, 2, 0, 1, 14, 0, 0, 2, 0]
[0, 97, 3, 0, 0, 8, 0, 0, 0, 0]
[2, 1, 75, 9, 2, 2, 5, 1, 5, 1]
[0, 0, 0, 80, 0, 12, 0, 7, 1, 0]
[0, 0, 0, 1, 93, 2, 2, 1, 3, 5]
[0, 0, 0, 4, 1, 83, 0, 2, 2, 0]
[2, 3, 3, 0, 6, 17, 58, 0, 2, 0]
[0, 4, 9, 0, 1, 5, 0, 82, 1, 4]
[1, 0, 2, 14, 0, 12, 0, 2, 70, 2]
[0, 0, 1, 5, 4, 6, 0, 4, 1, 79]
Label: 0 was accurate at a rate of: 78.88888888888889%
Label: 1 was accurate at a rate of: 89.81481481481481%
Label: 2 was accurate at a rate of: 72.81553398058253%
Label: 3 was accurate at a rate of: 80.0%
Label: 4 was accurate at a rate of: 86.91588785046729%
Label: 5 was accurate at a rate of: 90.21739130434783%
Label: 6 was accurate at a rate of: 63.73626373626374%
Label: 7 was accurate at a rate of: 77.35849056603773%
Label: 8 was accurate at a rate of: 67.96116504854369%
Label: 9 was accurate at a rate of: 79.0%
overall accuracy: 78.8% 

Perceptrons and their respective weight vectors were reset in between all the below test: 

no decay, I=0, B=0, NW, FO, #E=2 